<a href="https://colab.research.google.com/github/HoTrungQuan2004/ParkingAreaManage/blob/main/Train_License_plate_detection_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install tensorflow
!pip install tensorflow.keras

In [11]:
import os
import cv2
import numpy as np
from glob import glob
from tensorflow.keras import layers, models
import tensorflow as tf

In [14]:
from google import colab
colab.drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [16]:
def load_data(folder_path, img_size=96, orig_w=640, orig_h=410):
    img_dir = os.path.join(folder_path, 'images')
    label_dir = os.path.join(folder_path, 'labels')

    X, y = [], []

    for img_path in glob(os.path.join(img_dir, "*.jpg")):
        label_path = os.path.join(label_dir, os.path.basename(img_path).replace('.jpg', '.txt'))
        if not os.path.exists(label_path):
            continue

        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_size, img_size))
        with open(label_path) as f:
            line = f.readline().strip().split()
            if len(line) != 5:
                continue
            # Giả sử file label là: [class x y w h] (pixel gốc trên ảnh 640x410)
            _, x, y_, w, h = map(float, line)
            # Chuyển đổi bbox sang hệ ảnh 96x96
            x = x * img_size / orig_w
            y_ = y_ * img_size / orig_h
            w = w * img_size / orig_w
            h = h * img_size / orig_h
            # Chuẩn hóa về [0,1]
            x, y_, w, h = x / img_size, y_ / img_size, w / img_size, h / img_size
            X.append(img / 255.0)
            y.append([x, y_, w, h])

    return np.array(X), np.array(y)

In [17]:
X_train, y_train = load_data(
    "/content/drive/MyDrive/license_plate/dataset/plate_detection/train",
    img_size=96, orig_w=640, orig_h=410
)
X_val, y_val = load_data(
    "/content/drive/MyDrive/license_plate/dataset/plate_detection/val",
    img_size=96, orig_w=640, orig_h=410
)

In [18]:
# 🚀 Huấn luyện
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=16)

Epoch 1/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 11s 170ms/step - loss: 0.2496 - val_loss: 0.2419
Epoch 2/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 20s 163ms/step - loss: 0.2473 - val_loss: 0.2419
Epoch 3/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 21s 168ms/step - loss: 0.2516 - val_loss: 0.2419
Epoch 4/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 11s 164ms/step - loss: 0.2493 - val_loss: 0.2419
Epoch 5/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 20s 155ms/step - loss: 0.2517 - val_loss: 0.2419
Epoch 6/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 21s 169ms/step - loss: 0.2510 - val_loss: 0.2419
Epoch 7/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 8s 126ms/step - loss: 0.2514 - val_loss: 0.2419
Epoch 8/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 11s 174ms/step - loss: 0.2501 - val_loss: 0.2419
Epoch 9/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 18s 139ms/step - loss: 0.2533 - val_loss: 0.2419
Epoch 10/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 11s 146ms/step - loss: 0.2517 - val_loss: 0.2419
Epoch 11/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 13s 186ms/step - loss: 0.2537 - val_loss: 0.2419
Epoch 12/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 17s

In [19]:
#test model
X_test, y_test = load_data("/content/drive/MyDrive/license_plate/dataset/plate_detection/test")
loss = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.2478
Test loss: 0.2499302178621292


In [20]:
#show label (to check)
for i in range(5):
    print(y_train[i])

[0.29375    0.36097561 0.6515625  0.69512195]
[0.5703125  0.85853659 0.765625   0.92926829]
[0.36875    0.48780488 0.61875    0.58536585]
[0.403125   0.44878049 0.6390625  0.56341463]
[0.4140625  0.46341463 0.640625   0.76585366]


In [22]:
#convert into .tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open("plate_detector.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpww4p2gv_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  140268140304080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140268140304848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140268140306576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140268140304272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140268140306960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140268140307152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140268140304464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140268140306000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140268140309072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140268140310032: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [23]:
!xxd -i plate_detector.tflite > plate_model.h